In [1]:
# Version 72

In [2]:
def f1():
    # Just to initialize Global Variables
    return

In [3]:
def set_all_parameters():
    
    global epoc_used
    global mn             # Model Number - For essembling
    global batch_size
    global patience
    global nsplits
    global stop_split
    
    global num_lstm
    global num_dense
    global rate_drop_lstm
    global rate_drop_dense
    global rate_drop_spatial
    global loss
    global act
    global opt
    global met
    global es_mon
    global es_mode
    global Trainable
    global lr
    global method
    global model_list
    global maxlen
    global max_features
    global pretext_proc
    
    #
    # Choose Evaluate Method - Results change using different methods
    #
    #method="0" # Normal
    method="1" # EarlyStop F1
    #method="2" # Manual Epochs + Manual EarlyStop
    #method="3" # Manual Epochs + CLR
    #method="4" # EarlyStop F1 + CLR
    #method="5" # Manual Epochs + CLR + No EarlyStop

    model_list=[]
    model_list.append(["0",3,"Glove",1,"1"])
    
    ###
    ###  Load Train,Val and Test 
    ###
    maxlen = 70 # Maximum Sequence Size 
    max_features = 95000 # Maximum Number of Words in Dictionary
    pretext_proc=False
    
    epoc_used=0
    mn=-1
    ###
    ### Models Parameters
    ###
    batch_size=256 
    patience=1 #How many time insist running epochs after a bad result

    ###  4 Splits = Validation Split=0.25%
    ###  5 Splits = Validation Split=0.20 %
    ### 10 Splits = Validation Split=0.10 %
    ### 12 Splits = Validation Split=0.0833 %
    nsplits=4
    ### For Stratified k-fold - Validation -> nsplits=stop_splits
    stop_split=1   # Stop at split stop_split ina stratified k-fold
    num_lstm = 50
    num_dense = 50
    rate_drop_lstm = 0.05
    rate_drop_dense = 0.5
    rate_drop_spatial=0.05
    #loss="val_acc"
    #loss="val_loss"

    act = 'relu'
    ##
    ## Model Optimizer
    ##
    opt='adam'
    #opt='rmsprop'
    ##
    ## Model Metric
    ##
    #met=[f1]
    met=["accuracy"]  
    #met=["accuracy",f1]  
    ##
    ## Early Stop Monitor and Early Stop Mode
    ##
    #es_mon="val_f1"
    #es_mode="max" 
    #es_mon="val_loss"
    #es_mode="min"
    es_mon="val_acc"
    es_mode="max"
    Trainable=False
    lr=0.01

In [4]:
set_all_parameters()

**Import Libraries**

In [5]:
from IPython.display import display, HTML
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/embeddings"))

# Any results you write to the current directory are saved as output.
from tqdm import tqdm
import math
from datetime import timedelta
import time
from datetime import datetime
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten, Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, Reshape, Concatenate
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.layers import Bidirectional, GlobalMaxPool1D,GlobalMaxPooling1D,GlobalAveragePooling1D ,Conv1D, MaxPooling1D, GRU,CuDNNLSTM,CuDNNGRU, Reshape, MaxPooling1D,AveragePooling1D
from keras.optimizers import RMSprop, SGD, Nadam, Adamax, Adam
from keras import backend as K
from keras.engine.topology import Layer
#from keras import initializations
from keras import initializers, regularizers, constraints
from keras.layers import Conv2D, MaxPool2D
%matplotlib inline

['test.csv', 'embeddings', 'sample_submission.csv', 'train.csv']
['glove.840B.300d']


Using TensorFlow backend.


**Attention - Keras Layer**

In [6]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)
    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

[ https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate/code](http://)

In [7]:
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
    

**Function to Check Embeddings Coverage over Vocabulary**

In [8]:
import operator 
#for ele in t.word_counts:
#  print(ele,t.word_counts[ele])
def check_coverage(vocab,embeddings_index):
    
    try:
        print ("Len - embeddings_index:",len(embeddings_index))
    except :
        print ( "Len - embeddings_index:",len(embeddings_index.index2word))
    print ("Len -",len(vocab))
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:
            
            oov[word] = vocab[word]
            i += vocab[word]
            pass
    print ("Words Found:",k)
    print ("Words Not Found:",i)
    print ("Total Words:",i+k)
    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

**Function to Pre-Process Data**

In [9]:
########################################
## process texts in datasets
########################################
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

def ReplaceThreeOrMore(s):
    # pattern to look for three or more repetitions of any character, including
    # newlines.
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
    return pattern.sub(r"\1", s)

def splitstring(s):
    # searching the number of characters to split on
    proposed_pattern = s[0]
    for i, c in enumerate(s[1:], 1):
        if c != " ":
            if proposed_pattern == s[i:(i+len(proposed_pattern))]:
                # found it
                break
            else:
                proposed_pattern += c
    else:
        exit(1)

    return proposed_pattern

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x
#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)

#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

def text_to_wordlist(text,to_lower=False, rem_urls=False, rem_3plus=False,
                     clean_t=True, clean_num=True,mispelling=True,rem_specwords= False,
                     split_repeated=True, rem_special=False, rep_num=False, 
                     man_adj=True, rem_stopwords=False, stem_snowball=False,
                     stem_porter=False, lemmatize=False):

    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    if rem_urls:
        text = remove_urls(text)
    if to_lower:    
        text = text.lower()
    if rem_3plus:    
        text = ReplaceThreeOrMore(text)
        
    if clean_t:
        text= clean_text(text)
        
    if clean_num:
        text= clean_numbers(text)    
        
    if mispelling:
        text= replace_typical_misspell(text)

    if man_adj: 
        # Clean the text
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)

    # split them into a list
    text = text.split()
    
    if split_repeated:
        for i, c in enumerate(text):
            text[i]=splitstring(c)
    
    if rem_specwords:    
        to_remove = ['a','to','of','and']
        text = [w for w in text if not w in to_remove]
        
    # Optionally, remove stop words
    if rem_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    if rem_special: 
        text=special_character_removal.sub('',text)
    
    #Replace Numbers
    if rep_num:     
        text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_snowball:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    if stem_porter:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in text.split()])
        
    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w) for w in text.split()])   
 
    # Return a list of words
    return(text)

**Load Train and Tests Tables, Build the Vocbulary**
**Optional: Do Text Processing to clean data**

In [10]:
def load_and_prec(PreProcess=False):
    
    train_df = pd.read_csv("../input/train.csv")
    test_df = pd.read_csv("../input/test.csv")
    print("Train shape : ",train_df.shape)
    print("Test shape : ",test_df.shape)
    
    ## split to train and val
    #train_fit, train_val = train_test_split(train_df, test_size=0.08, random_state=2018)
    train_X=train_df["question_text"].values
    test_X=test_df["question_text"].values
    
    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    if PreProcess:
        train_questions = []
        for text in train_X:
            train_questions.append(text_to_wordlist(text))  
        test_questions=[]
        for text in test_X:
            test_questions.append(text_to_wordlist(text)) 
        tokenizer.fit_on_texts(train_questions+test_questions)   
        train_X = tokenizer.texts_to_sequences(train_questions)
        test_X = tokenizer.texts_to_sequences(test_questions) 
    else:
        tokenizer.fit_on_texts(list(train_X)+list(test_X))
        train_X = tokenizer.texts_to_sequences(train_X)
        test_X = tokenizer.texts_to_sequences(test_X)

    print(len(train_X), 'train sequences')
    print(len(test_X), 'test sequences')
    print('Average train sequence length: {}'.format(np.mean(list(map(len, train_X)), dtype=int)))
    print('Average test sequence length: {}'.format(np.mean(list(map(len, test_X)), dtype=int)))
    print('Max train sequence length: {}'.format(np.max(list(map(len, train_X)))))
    print('Max test sequence length: {}'.format(np.max(list(map(len, test_X)))))  
    
    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['target'].values
  
    print(len(tokenizer.word_index))
    print(len(tokenizer.word_counts))
    
    return train_X, test_X, train_y, test_df, tokenizer.word_index

**mdln="0-Bidirec(CuDNNGRU)-GlobalMax-Dense-Dropout"**

In [11]:
########################################
## BASE Model
########################################
def create_model0(embedding_matrix):

    mdln="0-Bidirec(CuDNNGRU)-GlobalMax-Dense-Dropout"
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
    
    x = LSTM(num_lstm, return_sequences=True)(embedding_layer)
    x = GlobalMaxPool1D()(x)
    x = Dense(num_dense, activation="relu")(x)
    x = Dropout(rate_drop_dense)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=met)
       
    return model, mdln

**mdln="1-LSTM-Dropout-Attention-Dense-Dropout-BatchNormalization"**

In [12]:
def create_model1(embedding_matrix):
    
    mdln="1-LSTM-Dropout-Attention-Dense-Dropout-BatchNormalization"
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)

    x = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences=True)(embedding_layer)
    x = Dropout(rate_drop_lstm)(x)
    x = Attention(maxlen)(x)
    x = Dense(num_dense, activation=act)(x)
    x = Dropout(rate_drop_dense)(x)
    x = BatchNormalization()(x)
    preds = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)
    
    return model, mdln

**    mdln="2-Bidirect(CuDNNLSTM)-GlobalMax-Dense-Dropout-Dense-Dropout"**

In [13]:
########################################
## BASE Model
########################################
def create_model2(embedding_matrix):

    mdln="2-Bidirect(CuDNNLSTM)-GlobalMax-Dense-Dropout-Dense-Dropout"
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)

    #x = Bidirectional(LSTM(num_dense, return_sequences=True, dropout=rate_drop_dense, recurrent_dropout=rate_drop_lstm))(embedding_layer)
    x = Bidirectional(CuDNNLSTM(num_lstm, return_sequences=True))(embedding_layer)
    x = GlobalMaxPool1D()(x)
    x = Dense(num_dense, activation="relu")(x)
    x = Dropout(rate_drop_dense)(x)
    x = Dense(num_dense, activation="relu")(x)
    x = Dropout(rate_drop_dense)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)
    
    return model, mdln

**  mdln="3-Bidirect(CuDNNLSTM)-Bidirect(CuDNNLSTM)-Attention-Dense"**

In [14]:
def create_model3(embedding_matrix):
    
    mdln="3-Bidirect(CuDNNLSTM)-Bidirect(CuDNNLSTM)-Attention-Dense"
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
    num_lstm2=int(num_lstm/2)
    x = Bidirectional(CuDNNLSTM(num_lstm, return_sequences=True))(embedding_layer)
    x = Bidirectional(CuDNNLSTM(num_lstm2, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    x = Dense(num_dense, activation="relu")(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=1e-3),metrics=met)

    return model, mdln

**mdln="4-Reshape-Concat(Conv2D(Filters)+MaxPool2d)-Flatten-Dropout"**

In [15]:
def create_model4( embedding_matrix):
    
    mdln="4-Reshape-Concat(Conv2D(Filters)+MaxPool2d)-Flatten-Dropout"
    
    filter_sizes = [1,2,3,5]
    num_filters = 36

    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
    x = Reshape((maxlen, embedding_matrix.shape[1], 1))(embedding_layer)
    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embedding_matrix.shape[1]),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.1)(z)

    preds = Dense(1, activation="sigmoid")(z)

    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)
    return model, mdln

** mdln="05-conv-max-conv-max-bulstm-max-dd"**

In [16]:
from keras.models import Sequential

def create_model5(embedding_matrix):

    mdln="05-conv-max-conv-max-bulstm-max-dd"
    
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)

    #x = Dropout(0.2)(embedded_sequences)
    #x = Conv1D(filters=64, kernel_size=5, padding='same', activation='relu')(x)
    x = Dropout(0.2)(embedding_layer)
    x = Conv1D(filters=embedding_matrix.shape[1], kernel_size=4, padding='same', activation='relu')(x)
    #x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Conv1D(filters=embedding_matrix.shape[1], kernel_size=4, padding='same', activation='relu')(x)
    #x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    #x = GRU(32)(main)
    #x = Dense(32, activation="relu")(x)
    #x = BatchNormalization()(x)
    #x = MaxPooling1D(pool_size=4)(x)
    #x = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences=True)(x)
    x = Bidirectional(CuDNNLSTM(num_lstm, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(num_dense, activation="relu")(x)
    x = Dropout(rate_drop_dense)(x)
    x = Dense(num_dense, activation="relu")(x)
    x = Dropout(rate_drop_dense)(x)
    preds = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)

    return model, mdln

** mdln="6-Bidirect(CuDNNGRU)-Cocat(GlobalMax+Global Avarage)- Dense - Dropout"**

In [17]:
def create_model6( embedding_matrix):

    mdln="6-Bidirect(CuDNNGRU)-Cocat(GlobalMax+Global Avarage)- Dense - Dropout"

    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)


    x = Bidirectional(CuDNNGRU(num_lstm, return_sequences=True))(embedding_layer)
  

    tower_1 = GlobalMaxPool1D()(x)
    tower_2 = GlobalAveragePooling1D()(x)
    
    output = concatenate([  tower_1, tower_2])

    x = Dense(num_dense, activation="relu")(output)
    x = Dropout(rate_drop_dense)(x)
    preds = Dense(1, activation="sigmoid")(x)                         

    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy',optimizer=opt,metrics=met)

   
    return model, mdln

**mdln="7-Bidirect(CuDNNGRU)-Attention-Dense-Dropout"**

In [18]:
def create_model7( embedding_matrix):
    
    mdln="7-Bidirect(CuDNNGRU)-Attention-Dense-Dropout"
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
    x = Bidirectional(CuDNNGRU(num_lstm, return_sequences=True))(embedding_layer)
    x = Attention(maxlen)(x) # New
    x = Dense(num_dense, activation="relu")(x)
    x = Dropout(rate_drop_dense)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=met)
    
    return model , mdln   

** mdln="8-Bidirect(CuDNNGRU)-Concat(GlobalMax+GlobalAverage)-Dense-Dropout"**

In [19]:
def create_model8( embedding_matrix):
    
    mdln="8-Bidirect(CuDNNGRU)-Concat(GlobalMax+GlobalAverage)-Dense-Dropout"    
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
    x = Bidirectional(CuDNNGRU(num_lstm, return_sequences=True))(embedding_layer)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(num_dense, activation="relu")(conc)
    conc = Dropout(rate_drop_dense)(conc)
    preds = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=met)
    
    return model, mdln

**    mdln="9-Bidirect(CuDNNGRU)-Bidirect(CuDNNGRU)-Bidirect(CuDNNGRU)-Attention-Dense"**

In [20]:
def create_model9(embedding_matrix):
    mdln="9-Bidirect(CuDNNGRU)-Bidirect(CuDNNGRU)-Bidirect(CuDNNGRU)-Attention-Dense"
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
    num_lstm2=int(num_lstm)
    num_lstm12=num_lstm2+int(num_lstm2/2)
    x = Bidirectional(CuDNNGRU(num_lstm, return_sequences=True))(embedding_layer)
    x = Bidirectional(CuDNNGRU(num_lstm12, return_sequences=True))(x)
    x = Bidirectional(CuDNNGRU(num_lstm2, return_sequences=True))(x)
    x = Attention(maxlen)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=met)
    
    return model, mdln

** mdln="14-Bidirect(CuDNNLSTM)-SpatialDropout-GlobalMax-BatchNorm-Dense-Dropout" **

In [21]:
def create_model14(embedding_matrix):

        mdln="14-Bidirect(CuDNNLSTM)-SpatialDropout-GlobalMax-BatchNorm-Dense-Dropout"      
        input_layer = Input(shape=(maxlen,))
        embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)

        # we add a GlobalMaxPool1D, which will extract information from the embeddings
        # of all words in the document
        x = CuDNNLSTM(num_lstm, return_sequences=True)(embedding_layer)
        x = SpatialDropout1D(rate_drop_spatial)(x)
        x = GlobalMaxPool1D()(x)

        # normalized dense layer followed by dropout
        x = BatchNormalization()(x)
        x = Dense(num_dense)(x)
        x = Dropout(rate_drop_dense)(x)

        # We project onto a six-unit output layer, and squash it with sigmoids:
        preds = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=input_layer, outputs=preds)
        model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)
                                         
        
        return model, mdln

**mdln="15-SpatialDropout-Bidirect(CudNNGRU)+Concat(GlobalMax+GlobalAverage)+Dense+Dropout"**

In [22]:
def create_model15(embedding_matrix):
### https://www.kaggle.com/yekenot/pooled-gru-fasttext
        mdln="15-SpatialDropout-Bidirect(CudNNGRU)+Concat(GlobalMax+GlobalAverage)+Dense+Dropout"
        input_layer = Input(shape=(maxlen,))
        embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
   
        x = SpatialDropout1D(rate_drop_spatial)(embedding_layer)
        x = Bidirectional(CuDNNGRU(num_lstm, return_sequences=True))(x)
        avg_pool = GlobalAveragePooling1D()(x)
        max_pool = GlobalMaxPooling1D()(x)        
        conc = concatenate([avg_pool, max_pool])
        x = Dense(num_dense, activation="relu")(conc)
        x = Dropout(rate_drop_dense)(x)
        preds = Dense(1, activation="sigmoid")(x)

        model = Model(inputs=input_layer, outputs=preds)
        model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)
     
        return model, mdln

** mdln="16-Bidirect(CudNNGRU)+GlobalMax+Dense+Dropout"**

In [23]:
def create_model16(embedding_matrix):

    mdln="16-Bidirect(CudNNGRU)+GlobalMax+Dense+Dropout"
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)

    x = Bidirectional(CuDNNLSTM(num_lstm, return_sequences=True))(embedding_layer)
    x = GlobalMaxPool1D()(x)
    x = Dense(num_dense, activation="relu")(x)
    x = Dropout(rate_drop_dense)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)
    
    return model, mdln

**        mdln="17-Bidirec(CuDNNLSTM)-GlobalMax-Dense-Dropout-Dense-Dropout"**

In [24]:
def create_model17(embedding_matrix ):
    
        mdln="17-Bidirec(CuDNNLSTM)-GlobalMax-Dense-Dropout-Dense-Dropout"
        input_layer = Input(shape=(maxlen,))
        embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
        x = Bidirectional(CuDNNLSTM(num_lstm, return_sequences=True))(embedding_layer)
        x = GlobalMaxPool1D()(x)
        x = Dense(num_dense, activation="relu")(x)
        x = Dropout(rate_drop_dense)(x)
        x = Dense(num_dense, activation="relu")(x)
        x = Dropout(rate_drop_dense)(x)
        preds = Dense(1, activation="sigmoid")(x)
        model = Model(inputs=input_layer, outputs=preds)
        model.compile(loss='binary_crossentropy',optimizer=opt, metrics=met)

        return model, mdln  

**mdln="18-SpatialDropout+Bidirect(CudNNGRU)+Bidirect(CudNLSTM)+Attention+Attention+Dense+Dropout"
** 

In [25]:
def create_model18(embedding_matrix):
    
    mdln="18-SpatialDropout+Bidirect(CudNNGRU)+Bidirect(CudNLSTM)+Attention+Attention+Dense+Dropout"
       
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                weights=[embedding_matrix], trainable=Trainable)(input_layer)
    x = SpatialDropout1D(rate_drop_spatial)(embedding_layer)
    x = Bidirectional(CuDNNLSTM(num_lstm, return_sequences=True))(x)
    y = Bidirectional(CuDNNGRU(num_lstm, return_sequences=True))(x)
    
    atten_1 = Attention(maxlen)(x) # skip connect
    atten_2 = Attention(maxlen)(y)
    avg_pool = GlobalAveragePooling1D()(y)
    max_pool = GlobalMaxPooling1D()(y)
    
    conc = concatenate([atten_1, atten_2, avg_pool, max_pool])
    conc = Dense(num_dense, activation="relu")(conc)
    conc = Dropout(rate_drop_dense)(conc)
    preds = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=input_layer, outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=met)
    
    return model,mdln

**mdln="19-SpatialDropout+Bidirect(CudNNGRU)+Conv1D+Concat(GlobalMax+GlobalAverage)+Dense+Dropout"
**

In [26]:
def create_model19( embedding_matrix):
    
    
        mdln="19-SpatialDropout+Bidirect(CudNNGRU)+Conv1D+Concat(GlobalMax+GlobalAverage)+Dense+Dropout"
        
        input_layer = Input(shape=(maxlen,))
        embedding_layer = Embedding(embedding_matrix.shape[0], embedding_matrix.shape[1],
                                    weights=[embedding_matrix], trainable=Trainable)(input_layer)

        x = SpatialDropout1D(rate_drop_spatial)(embedding_layer)
        x = Bidirectional(CuDNNGRU(num_lstm, return_sequences=True))(x)

        #x = Conv1D(filters=num_lstm, kernel_size=2, padding='same', activation='relu')(x)
        x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
        tower_1 = GlobalMaxPool1D()(x)
        tower_2 = GlobalAveragePooling1D()(x)

        output = concatenate([  tower_1, tower_2])

        out = Dense(num_dense, activation="relu")(output)
        out = Dropout(rate_drop_dense)(out)
        preds = Dense(1, activation="sigmoid")(out)                         

        model = Model(inputs=input_layer, outputs=preds)
        #model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=met)               


        return model, mdln    
        
       

**Auxiliary Funciton to Create the Models**

In [27]:
def create_model (label,embedding_matrix):
    
            if (label=="0"):
                return create_model0(embedding_matrix)
            if (label=="1"):
                return create_model1(embedding_matrix)
            if (label=="2"):
                return create_model2(embedding_matrix)
            if (label=="3"):
                return create_model3(embedding_matrix)            
            if (label=="4"):
                return create_model4(embedding_matrix)
            if (label=="5"):
                return create_model5(embedding_matrix)
            if (label=="6"):
                return create_model6(embedding_matrix)
            if (label=="7"):
                return create_model7(embedding_matrix)  
            if (label=="8"):
                return create_model8(embedding_matrix)
            if (label=="9"):
                return create_model9(embedding_matrix) 
            if (label=="10"):
                return create_model10(embedding_matrix)            
            if (label=="11"):
                return create_model11(embedding_matrix)                
            if (label=="12"):
                return create_model12(embedding_matrix)                
            if (label=="13"):
                return create_model13(embedding_matrix)                
            if (label=="14"):
                return create_model14(embedding_matrix)                
            if (label=="15"):
                return create_model15(embedding_matrix)                
            if (label=="16"):
                return create_model16(embedding_matrix)   
            if (label=="17"):
                return create_model17(embedding_matrix) 
            if (label=="18"):
                return create_model18(embedding_matrix) 
            if (label=="19"):
                return create_model19(embedding_matrix) 
            return None,"None"
            

**Funcion to calculate F1 Metric and Plot the Confusion Matrix**

In [28]:
from scikitplot.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
def best_F1 (v_y ,t_y,show_plot=False):
    bs=0
    bt=0
    for thresh in np.arange(0.1, 0.501, 0.01):
        thresh = np.round(thresh, 2)
        score = metrics.f1_score(v_y, (t_y>=thresh).astype(int))
        if score >= bs:
            bt = thresh
            bs = score  
    if (show_plot):        
        print("\n Best F1 score at threshold %2.4f is %2.4f \n" % (bt, bs))
        plot_confusion_matrix(v_y, np.array(pd.Series(t_y.reshape(-1,)).map(lambda x:1 if x>=bt else 0)))
        d=confusion_matrix(v_y, (t_y>=bt).astype(int))
        print ("Total:",d.sum(),"/Unsincere:",d[1,:].sum(),"/Sincere:",d[0,:].sum())
        print ("Total Erros:",(d[0,1]+d[1,0]),"/Unsincere: %2f"%((d[0,1]/(d[0,1]+d[1,1]))),"/Sincere: %2f" % ((d[1,0]/(d[1,0]+d[0,0]))))
        
    return bt, bs

**F1 Function to use in early stop callback - Evaluate Model 1 Function**

In [29]:
from keras import backend as K
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


'**Function to record results in a pandas dataframe**

In [30]:
def rec_results (Model,Embedding,Thresh,Score,StartTime):
    global resultsdf
    resultsdf = resultsdf.append({'Model':Model,
                                  'Method':method,
                                  'Embedding':Embedding,
                                  'Pretext':pretext_proc,
                                  'Weigth':wgt,
                                  'F1':Score,
                                  "Threshold":Thresh,
                                  "Duration":str(round((time.time()-StartTime)/60,0)),
                                  'Epochs':epochs,
                                  'EpochsUsed':epoc_used,
                                  'Trainable':Trainable,
                                  'Opt':opt,
                                  'MaxLength':maxlen,
                                  'MaxFeatures':max_features,                                  
                                  'batch_size':batch_size,
                                  'patience':patience,
                                  'num_lstm':num_lstm,
                                  'rate_drop_lstm':rate_drop_lstm,
                                  'num_dense':num_dense,
                                  'rate_drop_dense':rate_drop_dense,
                                  'rate_drop_spatial':rate_drop_spatial,
                                  'act':act,
                                  'opt':opt,
                                  'met':met,
                                  'es_mon':es_mon,
                                  'es_mode':es_mode,
                                  'Date':datetime.now().strftime("%d-%m-%Y %H:%M")}, ignore_index=True)

**Function to load Embedding files (Glove, Paragram, Wiki and Google)**

In [31]:
def load_embedding (emb,word_index) :
        estart = time.time()
        print('Indexing '+emb+' vectors')
        if (emb=="Glove"):
            EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
            def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
            embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
        if (emb=="Google"):
            EMBEDDING_FILE = '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
            embeddings_index = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
        if (emb=="Paragram"):
            EMBEDDING_FILE =  '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt' 
            def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
            embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)
        if (emb=="Wiki"):
            EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'    
            def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
            embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)
        
        print("Vector",EMBEDDING_FILE )
        print("End Indexing:",(str(timedelta(seconds=(time.time()-estart)))) )
        estart = time.time()
        print('Preparing embedding matrix')

        if (emb=="Wiki" or emb=="Glove" or emb=="Paragram"):
            all_embs = np.stack(embeddings_index.values())
            emb_mean,emb_std = all_embs.mean(), all_embs.std()
            embed_size = all_embs.shape[1]

            #word_index = tokenizer.word_index
            nb_words = min(max_features, len(word_index))+1
            embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
            for word, i in word_index.items():
                if i >= max_features: continue
                embedding_vector = embeddings_index.get(word)
                if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        else: #"Google"
            embed_size=300
            #word_index = tokenizer.word_index
            nb_words = min(max_features, len(word_index))+1
            embedding_matrix= (np.random.rand(nb_words, embed_size) - 0.5) / 5.0
            for word, i in word_index.items():
                if i >= max_features: continue
                if word in embeddings_index:
                    embedding_vector = embeddings_index.get_vector(word)
                    embedding_matrix[i] = embedding_vector
        print("End Preparing embedding matrix:",(str(timedelta(seconds=(time.time()-estart)))) )
        del embeddings_index
        return embedding_matrix

**Evaluate Model -Standart fit method**

In [32]:
def evaluate_model (label,emb, embedding_matrix,train_X,train_y,t_X, t_y, v_X, v_y, epochs, batch_size,patience):

        global resultsdf 
        global pred_val_ytrain_y
        global prev_test_y
        global met

        best_thresh = 0.0
        best_score = 0.0
        
        mstart = time.time()
        model=None
        model,mdln = create_model(label, embedding_matrix)
        print ("")
        print (mdln+'-'+emb)
        #print(modelx.summary())
        print("Model Fitting:")
        model.fit(t_X, t_y, batch_size=batch_size, epochs=epochs, validation_data=(v_X, v_y))
        print("Predicting Values:")
        pvy = model.predict([v_X], batch_size=1024, verbose=2)
        best_thresh,best_score=best_F1(v_y,  pvy[:,0] , False)
        print("\n")
        pty = model.predict([test_X], batch_size=1024, verbose=2)
        rec_results (mdln,emb,best_thresh,best_score,mstart)
        import gc;           
        del model
        gc.collect()
        time.sleep(10)  
        return pvy[:,0],pty[:,0],best_score,best_thresh

**Evaluate Model 1- Uses the early stop with F1 Metric **

In [33]:
def evaluate_model1 (label,emb, embedding_matrix,train_X,train_y,t_X,t_y,v_X,v_y, epochs, batch_size, patience=0):

        global resultsdf 
        global pred_val_y
        global prev_test_y
        global met 
        global epoc_used
        best_thresh = 0.0
        best_score = 0.0
        mstart = time.time()

        
        model=None
        model,mdln = create_model(label,embedding_matrix)
        print ("")
        print (mdln+'-'+emb)

        model_checkpoint = ModelCheckpoint('./model.hdf5', monitor=es_mon, mode=es_mode,
                                      verbose=True, save_best_only=True, 
                                      save_weights_only=False)
    
        early_stopping = EarlyStopping(monitor=es_mon, mode=es_mode, patience=patience,verbose=True)
        callbacks = [ early_stopping, model_checkpoint]
        print("Model Fitting:")
        hist= model.fit(t_X, t_y, batch_size=batch_size, epochs=epochs,
              shuffle=True, verbose=True, validation_data=(v_X, v_y),
              callbacks=callbacks)
        #print("History\n",hist.history.keys()) 
        #print("F1",hist.history['f1'])
        epoc_used=len(hist.history[es_mon])
        ### Getting the Best Model
        model.load_weights('./model.hdf5')   

        print("Predicting Values:")
        #predictions_valid = bestmodel.predict(X_valid.astype('float32'), batch_size=batch_size, verbose=2)
        pvy = model.predict([v_X], batch_size=batch_size, verbose=2)
        best_thresh,best_score=best_F1(v_y,pvy[:,0] , False)             
        pty = model.predict([test_X], batch_size=1024, verbose=2)
        rec_results (mdln,emb,best_thresh,best_score,mstart)
        import gc;           
        del model
        gc.collect()
        time.sleep(10)
        return  pvy[:,0],pty[:,0],best_score,best_thresh

**Evaluate Model 2- Manualy generates epochs and uses F1 metric to early stop **

In [34]:
def evaluate_model2 (label,emb, embedding_matrix,train_X,train_y, t_X, t_y, v_X, v_y, epochs, batch_size, patience=0):
    
    global resultsdf 
    global pred_val_y
    global prev_test_y
    global met

    best_thresh = 0.0
    best_score = 0.0

    mstart = time.time()
    model=None
    model,mdln = create_model(label, embedding_matrix)
    print ("")
    print (mdln+'-'+emb)
    bs=0
    pt=0
    print("Model Fitting:")
    for e in range(epochs):
        model.fit(t_X, t_y, batch_size=batch_size, epochs=1, validation_data=(v_X, v_y))
        print("Predicting Values:")
        pvy = model.predict([v_X], batch_size=1024, verbose=0)
        best_thresh,best_score=best_F1(v_y,pvy[:,0], False )
        print("epoch ",e+1,"/",epochs)
        if (best_score > bs):          
            print("F1 Score Improved from %2.4f to %2.4f" % (bs,best_score))
            bs = best_score
            bt = best_thresh
            model.save_weights('./model.hdf5') 
        else:
            print("F1 Score not improved")
            pt=pt+1
            if (pt>patience):
                break
    model.load_weights('./model.hdf5')             
    pty = model.predict([test_X], batch_size=1024, verbose=0)
    rec_results (mdln,emb,bt,bs,mstart) 
    import gc;           
    del model
    gc.collect()
    time.sleep(10)
    return pvy[:,0],pty[:,0],bs,bt

**Evaluate Model -Using CLR method**

In [35]:
def evaluate_model3 (label,emb, embedding_matrix,train_X,train_y, t_X, t_y, v_X, v_y, epochs, batch_size, patience=0):
    
    global resultsdf 
    global pred_val_y
    global prev_test_y
    global met
    global clr
    global epoc_used 
    clr = CyclicLR(base_lr=0.001, max_lr=0.01,
               step_size=300., mode='exp_range',
               gamma=0.99994)
    callback = [clr,]

    best_thresh = 0.0
    best_score = 0.0

    mstart = time.time()
    model=None
    model,mdln = create_model(label, embedding_matrix)
    print ("")
    print (mdln+'-'+emb)
    bs=0
    pt=0
    print("Model Fitting:")
    for e in range(epochs):
        model.fit(t_X, t_y, batch_size=batch_size, epochs=1, validation_data=(v_X, v_y),callbacks = callback,verbose=2)
        print("Predicting Values:")
        pvy = model.predict([v_X], batch_size=1024, verbose=0)
        print("F1 - Score:")
        best_thresh,best_score=best_F1(v_y,pvy[:,0] )
        print("epoch ",e+1,"/",epochs)
        epoc_used=e
        if (best_score > bs):          
            print("F1 Score Improved from %2.4f to %2.4f" % (bs,best_score))
            bs = best_score
            bt = best_thresh
            model.save_weights('./model.hdf5') 
        else:
            print("F1 Score not improved")
            pt=pt+1
            if (pt>patience):
                epoc_used=epoc_used-1
                break
    model.load_weights('./model.hdf5')             
    pty = model.predict([test_X], batch_size=1024, verbose=0)
    rec_results (mdln,emb,bt,bs,mstart) 
    import gc;           
    del model
    gc.collect()
    time.sleep(10)
    return pvy[:,0],pty[:,0],bs,bt

Method EarlyStop + CLR****

In [36]:
def evaluate_model4 (label,emb, embedding_matrix,train_X,train_y,t_X,t_y,v_X,v_y, epochs, batch_size, patience=0):

        global resultsdf 
        global pred_val_y
        global prev_test_y
        global met 
        global clr
        global epoc_used 
        clr = CyclicLR(base_lr=0.001, max_lr=0.01,
                   step_size=300., mode='exp_range',
                   gamma=0.99994)
        best_thresh = 0.0
        best_score = 0.0
        
        mstart = time.time()
        model=None
        model,mdln = create_model(label,embedding_matrix)
        print ("")
        print (mdln+'-'+emb)
        model_checkpoint = ModelCheckpoint('./model.hdf5', monitor=es_mon, mode=es_mode,
                                      verbose=True, save_best_only=True, 
                                      save_weights_only=False)
    
        early_stopping = EarlyStopping(monitor=es_mon, mode=es_mode, patience=patience,verbose=True)
        callbacks = [ early_stopping, model_checkpoint,clr,]
        print("Model Fitting:")
        hist= model.fit(t_X, t_y, batch_size=batch_size, epochs=epochs,
              shuffle=True, verbose=True, validation_data=(v_X, v_y),
              callbacks=callbacks)
        epoc_used=len(hist.history[es_mon])  
        ### Getting the Best Model
        print ("Getting the Best Model")  
        model.load_weights('./model.hdf5')   

        print("Predicting Values:")
        #predictions_valid = bestmodel.predict(X_valid.astype('float32'), batch_size=batch_size, verbose=2)
        pvy = model.predict([v_X], batch_size=batch_size, verbose=2)
        best_thresh,best_score=best_F1(v_y,pvy[:,0], False )             
        pty = model.predict([test_X], batch_size=1024, verbose=2)
        rec_results (mdln,emb,best_thresh,best_score,mstart)
        import gc;           
        del model
        gc.collect()
        time.sleep(10)
        return  pvy[:,0],pty[:,0],best_score,best_thresh

In [37]:
def evaluate_model5 (label,emb, embedding_matrix,train_X,train_y, t_X, t_y, v_X, v_y, epochs, batch_size, patience=0):
    
    global resultsdf 
    global pred_val_y
    global prev_test_y
    global met
    global clr
    global epoc_used 
    clr = CyclicLR(base_lr=0.001, max_lr=0.01,
               step_size=300., mode='exp_range',
               gamma=0.99994)
    callback = [clr,]
    best_thresh = 0.0
    best_score = 0.0

    mstart = time.time()
    model=None
    model,mdln = create_model(label, embedding_matrix)
    print ("")
    print (mdln+'-'+emb)
    bs=0
    pt=0
    print("Model Fitting:")
    for e in range(epochs):
        model.fit(t_X, t_y, batch_size=batch_size, epochs=1, validation_data=(v_X, v_y),verbose=2)
        print("Predicting Values:")
        pvy = model.predict([v_X], batch_size=1024, verbose=0)
        best_thresh = 0.0
        best_score = 0.0
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            score = metrics.f1_score(v_y, (pvy[:,0]  > thresh).astype(int))
            if score > best_score:
                print("F1COnf Score Improved from %2.4f to %2.4f" % (best_score,score))
                best_thresh = thresh
                best_score = score
        best_thresh = 0.0
        best_score = 0.0
        print("F1 - Score:")
        best_thresh,best_score=best_F1(v_y,pvy[:,0] )
        print("epoch ",e+1,"/",epochs)
        epoc_used=e
        if (best_score > bs):          
            print("F1 Score Improved from %2.4f to %2.4f" % (bs,best_score))
            bs = best_score
            bt = best_thresh
            
    pty = model.predict([test_X], batch_size=1024, verbose=0)
    rec_results (mdln,emb,bt,bs,mstart) 
    import gc;           
    del model
    gc.collect()
    time.sleep(10)
    return pvy[:,0],pty[:,0],bs,bt

**Evaluate Model -Strafied K-Fold**

In [38]:
def evaluate_stratifiedkfold (label,mt,emb,train_X,train_y, test_X, n_splits, stop_split, epochs, batch_size,patience):
    
    global resultsdf 
    global allpred_val_y
    global allprev_test_y
    global mn
    global method
    method=mt
    random_seed = 2018
    train_stratified=np.zeros([len(train_X),(nsplits)])
    test_stratified = np.zeros([len(test_X),(nsplits)])
    msstart = time.time()
    embedding_matrix=embedding_matrix_combined
    if  (emb=="Glove"):
        embedding_matrix=embedding_matrix_1
    if  (emb=="Wiki"):
        embedding_matrix=embedding_matrix_2
    if  (emb=="Paragram"):
        embedding_matrix=embedding_matrix_3
    if  (emb=="Google"):
        embedding_matrix=embedding_matrix_4  

    splits = list(StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed).split(train_X, train_y))
    mn=mn+1
    bs=0
    bt=0
    for idx, (train_idx, valid_idx) in enumerate(splits):
           
            X_split = train_X[train_idx]
            y_split = train_y[train_idx]
            print ("Split:",idx)
            print ("Total X-train:",len(y_split)," Unsincere:",y_split.sum(),y_split.sum()/ len(y_split) )
            X_val = train_X[valid_idx]
            y_val = train_y[valid_idx]
            print ("Total X-val:",len(y_val)," Unsincere:",y_val.sum(), y_val.sum()/len(y_val))
            if (mt=="0"):
                pred_train_y, pred_test_y, best_score, best_thresh=evaluate_model(md,emb,embedding_matrix,train_X,train_y, X_split, y_split, X_val, y_val, epochs, batch_size ,patience)
            if (mt=="1"):
                pred_train_y, pred_test_y, best_score, best_thresh=evaluate_model1(md,emb,embedding_matrix,train_X,train_y, X_split, y_split, X_val, y_val,epochs, batch_size ,patience)
            if (mt=="2"):
                pred_train_y, pred_test_y, best_score, best_thresh=evaluate_model2(md,emb,embedding_matrix,train_X,train_y, X_split, y_split, X_val, y_val,epochs, batch_size ,patience) 
            if (mt=="3"):
                pred_train_y, pred_test_y, best_score, best_thresh=evaluate_model3(md,emb,embedding_matrix,train_X,train_y, X_split, y_split, X_val, y_val,epochs, batch_size ,patience) 
            if (mt=="4"):
                pred_train_y, pred_test_y, best_score, best_thresh=evaluate_model4(md,emb,embedding_matrix,train_X,train_y, X_split, y_split, X_val, y_val,epochs, batch_size ,patience) 
            if (mt=="5"):
                pred_train_y, pred_test_y, best_score, best_thresh=evaluate_model5(md,emb,embedding_matrix,train_X,train_y, X_split, y_split, X_val, y_val,epochs, batch_size ,patience) 
            if (best_score > bs):          
                print("F1 Score Improved from %2.4f to %2.4f" % (bs,best_score))
                bs = best_score
                bt = best_thresh
            #train_stratified[:,idx] =  pred_train_y
            test_stratified[:,idx] = pred_test_y
            if ((idx+1)==stop_split):
                break
    
    if (n_splits==stop_split):
        div=n_splits
    else:
        div=stop_split
    train_pred =  train_stratified.sum(axis=1)/div
    test_pred  = test_stratified.sum(axis=1)/div
    #train_pred =  train_stratified.max(axis=1)
    #test_pred  = test_stratified.max(axis=1)
    #print("F1 - Score - K-Fold:")
    #best_thresh,best_score=best_F1(train_y,train_pred,False )
    if (stop_split > 1):
       rec_results ("Model:"+label+"-skfold-"+str(n_splits)+"Stop:"+str(stop_split),emb,bt,bs,msstart)
    
    return train_pred,test_pred,best_score,best_thresh

**Load Train,Validation and Test Data. 
With or without pre-preprocessing.**

In [39]:
###
###  Load Train,Val and Test 
###

train_X, test_X, train_y, test_df, word_index = load_and_prec(True)

Train shape :  (1306122, 3)
Test shape :  (56370, 2)
1306122 train sequences
56370 test sequences
Average train sequence length: 12
Average test sequence length: 12
Max train sequence length: 128
Max test sequence length: 81
192940
192940


**Load Embedding Matrix from Glove,Wki,Paragram and Google. **
Create Combination of Matrix

In [40]:
###
### Load Embedding Matrix
###
embedding_matrix_1 = load_embedding("Glove",word_index)
#embedding_matrix_2 = load_embedding("Wiki",word_index)
#embedding_matrix_3 = load_embedding("Paragram",word_index)
#embedding_matrix_4 = load_embedding("Google",word_index)
## Simple average: http://aclweb.org/anthology/N18-2031

# We have presented an argument for averaging as
# a valid meta-embedding technique, and found experimental
# performance to be close to, or in some cases 
# better than that of concatenation, with the
# additional benefit of reduced dimensionality  


## Unweighted DME in https://arxiv.org/pdf/1804.07983.pdf

# “The downside of concatenating embeddings and 
#  giving that as input to an RNN encoder, however,
#  is that the network then quickly becomes inefficient
#  as we combine more and more embeddings.”
  
# embedding_matrix = np.mean([embedding_matrix_1, embedding_matrix_2, embedding_matrix_3], axis = 0)
embedding_matrix_combined = np.mean([embedding_matrix_1, embedding_matrix_1], axis = 0)
np.shape(embedding_matrix_combined)

Indexing Glove vectors
Vector ../input/embeddings/glove.840B.300d/glove.840B.300d.txt
End Indexing: 0:05:42.772082
Preparing embedding matrix
End Preparing embedding matrix: 0:00:09.717206


(95001, 300)

**Build Models List  to be Processed** 
**Create reh Results Array**

**model_list.append["Model", epochs,"Embedding",weight , evaluation_method]**

**Possible models ["0","1","2","3","4","5","6","7","8","9","14","15","16","17","18","19"]**

**Possible embeddings ["Glove","Paragram","Google","Wiki","Combined"]**

**Each Weight will be applied and results will be divided by the sum of weigths**

**method="0" # Normal**

**method="1" # EarlyStop F1**

**method="2" # Manual Epochs**

**method="3" # CLR**

**method="4" # EarlyStop F1 + CLR**

**Run Models in the Model List**
**List of Parameters to the Model Run**

In [41]:
###
### Run All Models
###
set_all_parameters()
bstart = time.time()
resultsdf = pd.DataFrame(columns=['Model','F1','Embedding','Pretext','Weigth','Duration','Method',
                                  'Epochs','EpochsUsed','patience','batch_size'
                                  ,'MaxLength','MaxFeatures' ])


### Matrix of Predictions - Validation and Test
### Stores All Predicted Values

allpred_train_y=np.zeros([len(train_X),(len(model_list))])
allpred_test_y=np.zeros([len(test_X),(len(model_list))])
allbest=np.zeros([(len(model_list))])
allthresh=np.zeros([(len(model_list))])


for ne,item in  enumerate(model_list): 
    emb=item[2]
    epochs=item[1]
    wgt=item[3]
    md=item[0] 
    mt=item[4]
    print (mt)
    print ("md-"+md)
    trainp,testp,bs,bt=evaluate_stratifiedkfold (md,mt,emb,train_X,train_y, test_X, nsplits, stop_split, epochs, batch_size,patience)
    allbest[mn]=bs
    allthresh[mn]=bt
    #allpred_train_y[:,mn] = trainp
    allpred_test_y[:,mn] = testp
    print ("FIM-"+md)



1
md-0
Split: 0
Total X-train: 979591  Unsincere: 60607 0.061869698680367624
Total X-val: 326531  Unsincere: 20203 0.06187161402745834

0-Bidirec(CuDNNGRU)-GlobalMax-Dense-Dropout-Glove
Model Fitting:
Train on 979591 samples, validate on 326531 samples
Epoch 1/3
979591/979591 [==============================] - 1499s 2ms/step - loss: 0.1240 - acc: 0.9541 - val_loss: 0.1088 - val_acc: 0.9574

Epoch 00001: val_acc improved from -inf to 0.95739, saving model to ./model.hdf5
Epoch 2/3
979591/979591 [==============================] - 1535s 2ms/step - loss: 0.1076 - acc: 0.9583 - val_loss: 0.1091 - val_acc: 0.9558

Epoch 00002: val_acc did not improve from 0.95739
Epoch 00002: early stopping
Predicting Values:
F1 Score Improved from 0.0000 to 0.6511
FIM-0


**Calculate weights and apply to results**

In [42]:
#### Aplicando Pesos nos Modelos
twgt=0
wgt_test_y=0
wgt_train_y=0
essemblet="Essemble: "
for ne,item in  enumerate(model_list): 
    twgt=twgt+item[3]
    #wgt_train_y=wgt_train_y+(allpred_train_y[:,ne]*item[3])
    wgt_test_y =wgt_test_y +(allpred_test_y[:,ne]*item[3])
    essemblet=essemblet+item[0]+" "
#wgt_train_y=wgt_train_y/twgt
wgt_test_y=wgt_test_y/twgt


#best_thresh,best_score=best_F1(train_y,wgt_train_y,True)
#rec_results (essemblet,'Essemble',best_thresh,best_score,bstart)
display (resultsdf)

,Model,F1,Embedding,Pretext,Weigth,Duration,Method,Epochs,EpochsUsed,patience,...,act,es_mode,es_mon,met,num_dense,num_lstm,opt,rate_drop_dense,rate_drop_lstm,rate_drop_spatial
0,0-Bidirec(CuDNNGRU)-GlobalMax-Dense-Dropout,0.651061,Glove,False,1,54.0,1,3,2,1,...,relu,max,val_acc,[accuracy],50.0,50.0,adam,0.5,0.05,0.05


**Write Output to submission file**

In [43]:
print (allthresh)
print (allthresh.min())
print (allthresh.mean())

# Write the output
y_te = (np.array(wgt_test_y) >= 0.33).astype(np.int)
submit_df = pd.DataFrame({"qid": test_df["qid"], "prediction": y_te})
submit_df.to_csv("./submission.csv",index=False)
display(submit_df)

[0.32]
0.32
0.32


,qid,prediction
0,00014894849d00ba98a9,0
1,000156468431f09b3cae,0
2,000227734433360e1aae,0
3,0005e06fbe3045bd2a92,0
4,00068a0f7f41f50fc399,0
5,000a2d30e3ffd70c070d,0
6,000b67672ec9622ff761,0
7,000b7fb1146d712c1105,0
8,000d665a8ddc426a1907,0
9,000df6fd2229447b2969,0
